In [1]:
'''
The purpose of this note book is to automate point cloud to Gtif in a more python friendly way.
The goal is to use some of the prexisting functions in abm_detect to streamline this.

Otherwise its just good ol:
pdal pipeline script.json


from the comand line...
which is really REALLY fast... but hard to experiment with :(
'''


'\nThe purpose of this note book is to automate point cloud to Gtif in a more python friendly way.\nThe goal is to use some of the prexisting functions in abm_detect to streamline this.\n\nOtherwise its just good ol:\npdal pipeline script.json\n\n\nfrom the comand line...\nwhich is really REALLY fast... but hard to experiment with :(\n'

In [2]:
import pdal
import ipyleaflet
import IPython.display
import ipyvolume.pylab as p3
import matplotlib.cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely.geometry
import scipy.spatial

import os
import requests
import json
import sys
import urllib.request


In [3]:
### Important directory paths

TM_las = '/home/jose/Documents/LiDAR/TM/input_LAS/points.laz'
py_export = '/home/jose/Github/my_projs/abm_detect/Figures/python_export/TM_lidar'

In [4]:
m = ipyleaflet.Map(center=(32.29, 253.3), zoom=13)
m

Map(center=[32.29, 253.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [ ]:
# IS USED FOR POLYGONS! not really needed seing as ipyleaflet uses osm. only see da roads :(
dc = ipyleaflet.DrawControl()
m.add_control(dc)

In [ ]:
dc.last_draw['geometry']['coordinates'][0]  # the coordinates of the drawn polygon

In [ ]:
# Takes 1hr and 17 mins to run 299338174
cropper = {
    "pipeline": [ TM_las,
        {   "type":"filters.eigenvalues",
            "knn":16},
        {   "type":"filters.normal",
            "knn":16}
    ]}
pipeline = pdal.Pipeline(json.dumps(cropper))
pipeline.validate()
%time n_points = pipeline.execute()
print('Pipeline selected {} points'.format(n_points))

In [ ]:
#creating numpy and pandas objects from pdal pipeline
arr = pipeline.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})
df['X_0'] = df['X']
df['Y_0'] = df['Y']
df['Z_0'] = df['Z']
df['X'] = df['X'] - df['X_0'].min()
df['Y'] = df['Y'] - df['Y_0'].min()
df['Z'] = df['Z'] - df['Z_0'].min()

In [ ]:
598007.598339 	  Ymin = 5059576.691053 	  Xmax = 600535.442345 	  Ymax = 5060126.433775

In [ ]:
'''
Reading into memory a point cloud las file and computing the eigenvalues and normal vectors.
  - Returns an array and descriptions which can later be used to generate pandas dataframes
'''


# reading into memory a point cloud las file and computing the eigenvalues and normal vectors.
# extracting array
def get_las(input_f,knn_int):
    
    def pipeline(json_obj):
        pipeline = pdal.Pipeline(json_obj)
        pipeline.validate()
        time n_points = pipeline.execute()
        print('Pipeline selected {} points'.format(n_points))
        pipeline_obj = pipeline
        
        return pipeline_obj
    
    pipeline_dict = {
        "pipeline": [ 
            input_f,
            {   "type":"filters.eigenvalues",
                "knn":knn_int},
            {   "type":"filters.normal",
                "knn":knn_int}
        ]
    }
    
    json_obj = json.dumps(pipeline_dict)
    pdal_obj = pipeline(json_obj)
    return pdal_obj.arrays[0], pdal_obj.arrays[0].dtype.descr
    

a = get_las( TM_las, knn_int = 16 )
data, description = a[0], a[1]

In [ ]:

cols = [col for col, __ in description]
df = pd.DataFrame({col: data[col] for col in cols})
df['X_0'] = df['X']
df['Y_0'] = df['Y']
df['Z_0'] = df['Z']
df['X'] = df['X'] - df['X_0'].min()
df['Y'] = df['Y'] - df['Y_0'].min()
df['Z'] = df['Z'] - df['Z_0'].min()

In [ ]:
a[1]

In [11]:
def geom_calc(las_path, new_path, knn_int):
    json_obj = {
        "pipeline": [
            las_path,
            {
               "type":"filters.eigenvalues",
                "knn":knn_int},

            {  "type":"filters.normal",
                "knn":knn_int},

            {
                "type":"writers.bpf",
                "filename":new_path
            }
        ]

    }
    
    pipeline = pdal.Pipeline(json.dumps(json_obj))
    pipeline.validate()
    n_points = pipeline.execute()
    print('Pipeline selected {} points'.format(n_points))
    
    
    

In [12]:
las_path = TM_las
new_path = py_export + '/' + 'TMall.bpf'

geom_calc(las_path, new_path, knn_int=16)

Pipeline selected 299338174 points


In [20]:

def bpf2tiff(bpf_path, write_path,dimension):
    json_obj ={
        "pipeline": [
            bpf_path,

        {
                "resolution": 1,
                "radius": 1.41,
                "dimension": "curvature"
                "gdaldriver": "Gtiff",
                "filename": write_path

        }


        ]
    }
    
    pipeline = pdal.Pipeline(json.dumps(json_obj))
    pipeline.validate()
    n_points = pipeline.execute()
    print('Pipeline selected {} points'.format(n_points))


    
#bpf2tiff(bpf_path, write_path,dimension)

In [21]:
bpf_path=new_path
write_path=py_export + '/' + 'TMall_curvature'
dimension='Curvature'

bpf2tiff(bpf_path, write_path,dimension)

RuntimeError: writers.text: Unexpected argument 'gdaldriver'.

In [22]:
bpf_path

'/home/jose/Github/my_projs/abm_detect/Figures/python_export/TM_lidar/TMall.bpf'